# Process calcium imaging data with DataJoint Elements

This notebook will walk through processing two-photon calcium imaging data collected
from ScanImage and processed with Suite2p.

The DataJoint Python API and Element Calcium Imaging offer a lot of features to support collaboration, automation, reproducibility, and visualizations.

For more information on these topics, please visit our documentation: 
 
- [DataJoint Core](https://datajoint.com/docs/core/): General principles

- DataJoint [Python](https://datajoint.com/docs/core/datajoint-python/) and
  [MATLAB](https://datajoint.com/docs/core/datajoint-matlab/) APIs: in-depth reviews of
  specifics

- [DataJoint Element Calcium Imaging](https://datajoint.com/docs/elements/element-calcium-imaging/):
  A modular pipeline for calcium imaging analysis

Let's start by importing the packages necessary to run this workflow. 

In [ ]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
import datetime
import matplotlib.pyplot as plt
import numpy as np

### The Basics:

Any DataJoint workflow can be broken down into basic 3 parts:

- `Insert`
- `Populate` (or process)
- `Analyze`

In this demo we will:
- `Insert` metadata about an animal subject, recording session, and 
  parameters related to processing calcium imaging data through Suite2p.
- `Populate` tables with outputs of image processing including motion correction,
  segmentation, mask classification, fluorescence traces and deconvolved activity traces.
- `Analyze` the processed data by ***querying*** and plotting activity traces.

Each of these topics will be explained thoroughly in this notebook.

### Workflow diagram

This workflow is assembled from 4 DataJoint elements:
+ [element-lab](https://github.com/datajoint/element-lab)
+ [element-animal](https://github.com/datajoint/element-animal)
+ [element-session](https://github.com/datajoint/element-session)
+ [element-calcium-imaging](https://github.com/datajoint/element-calcium-imaging)

Each element declares its own schema in the database. These schemas can be imported like
any other Python package. This workflow is composed of schemas from each of the Elements
above and correspond to a module within `workflow_calcium_imaging.pipeline`.

The schema diagram is a good reference for understanding the order of the tables
within the workflow, as well as the corresponding table type.
Let's activate the elements and view the schema diagram.

In [ ]:
from workflow_calcium_imaging.pipeline import lab, subject, session, scan, imaging, Equipment

In [ ]:
(
    dj.Diagram(subject.Subject)
    + dj.Diagram(session.Session)
    + dj.Diagram(scan)
    + dj.Diagram(imaging)
)

While the diagram above seems complex at first, it becomes more clear when it's
approached as a hierarchy of tables that define the order in which the workflow expects to
receive data in each of its tables. 

+ Tables with a green, or rectangular shape expect to receive data manually using the
`insert()` function. The tables higher up in the diagram such as `subject.Subject()`
should be the first to receive data. This ensures data integrity by preventing orphaned
data within DataJoint schemas. 
+ Tables with a purple oval or red circle can be automatically filled with relevant data
  by calling `populate()`. For example `scan.ScanInfo` and its part-table
  `scan.ScanInfo.Field` are both populated with `scan.ScanInfo.populate()`.


## Starting the workflow

### Insert entries into manual tables and populate automated tables

To view details about a table's dependencies and attributes, use functions `.describe()`
and `.heading`, respectively.

Let's start with the first table in the schema diagram (the `subject` table) and view
the table attributes we need to insert. There are two ways you can do this: *run each
of the two cells below*

In [ ]:
subject.Subject.describe()

In [ ]:
subject.Subject.heading

The cells above show all attributes of the subject table. These are particularly useful functions if you are new to
DataJoint Elements and are unsure of the attributes required for each table. We will insert data into the
`subject.Subject` table. 

In [ ]:
subject.Subject.insert1(
    dict(
        subject="subject1",
        sex="F",
        subject_birth_date="2020-01-01",
        subject_description="ScanImage acquisition. Suite2p processing.",
    )
)

Let's repeat the steps above for the `Session` table and see how the output varies between
`.describe` and `.heading`. 

In [ ]:
session.Session.describe()

In [ ]:
session.Session.heading

The cells above show the dependencies and attributes for the `session.Session` table.
Notice that `describe` shows the dependencies of the table on upstream tables. The
`Session` table depends on the upstream `Subject` table. Whereas `heading` lists all the
attributes of the `Session` table, regardless of whether they are declared in an upstream
table. 


Here we will demonstrate a very useful way of inserting data by assigning the dictionary
to a variable `session_key`. This variable can be used to insert entries into tables that
contain the `Session` table as one of its attributes.

In [ ]:
session_key = dict(subject="subject1", session_datetime="2021-04-30 12:22:15.032")

session.Session.insert1(session_key)

In [ ]:
session.SessionDirectory.insert1(
    dict(**session_key,
        session_dir="subject1/session1")
)

Next, we'll use `describe` and `heading` for the Scan table. Do you notice anything we
might have missed here? 

In [ ]:
scan.Scan.describe()

In [ ]:
scan.Scan.heading

The `Scan` table's attributes include the `Session` table **and** the `Equipment` table.
Let's see what happens when you try to insert data into `Scan` without inserting into
`Equipment` first.

In [ ]:
scan.Scan.insert1(
    dict(
        session_key,
        scanner="ScanImage",
        acq_software="ScanImage",
        scan_notes="",
    )
)

# This cell will produce an error

DataJoint issues an error because there is no entry in the `Equipment` table. Let's go
ahead and insert into the Equipment table after checking its attributes. Then, we will
try the insert into `Scan` again.

In [ ]:
Equipment.describe()

In [ ]:
Equipment.heading

In [ ]:
Equipment.insert1(dict(scanner="ScanImage"))

In [ ]:
scan.Scan.insert1(
    dict(
        **session_key,
        scan_id=0,
        scanner="ScanImage",
        acq_software="ScanImage",
        scan_notes="",
    )
)

This time, we were able to successfully insert data into the scan table. Now we're ready
to use DataJoint's automation features to populate several downstream tables after
`Scan`.

### Automatically populate tables

`scan.ScanInfo` is the first table in the pipeline that can be populated automatically.
If a table contains a part table, this part table is also populated during the
`populate()` call. Let's view the `scan.ScanInfo` and its part table
`scan.ScanInfo.Field`. 

In [ ]:
scan.ScanInfo.describe();

In [ ]:
scan.ScanInfo.heading

In [ ]:
scan.ScanInfo()

In [ ]:
scan.scanInfo.Field()

`populate()` takes a few arguments which we will set in the cell below and use it on `ScanInfo`.

In [ ]:
populate_settings = {
                    "display_progress": True,
                    "reserve_jobs": False,
                    "suppress_errors": False,
                    }

In [ ]:
# duration depends on your network bandwidth to s3
scan.ScanInfo.populate(**populate_settings)

Let's view that was entered into each of these tables:

In [ ]:
scan.scanInfo()

In [ ]:
scan.ScanInfo.Field()

We're almost ready to perform image processing with `Suite2p`. An important step before
processing is managing the parameters which will be used in this step. To do so, we will
import suite2p and insert the default parameters into a DataJoint table
`ProcessingParamSet`. This table keeps track of all your image processing
parameters. You can choose which parameters are used during processing in a later table.

Let's view the attributes and insert data into `imaging.ProcessingParamSet`.

In [ ]:
imaging.ProcessingParamSet.describe();

In [ ]:
imaging.ProcessingParamSet.heading

In [ ]:
import suite2p

params_suite2p = suite2p.default_ops()
params_suite2p['nonrigid']=False

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_idx=0,
    params=params_suite2p,
    paramset_desc="Calcium imaging analysis with Suite2p using default Suite2p parameters",
)

Now that we've inserted suite2p parameters into the `ProcessingParamSet` table,
we're almost ready to run image processing. DataJoint uses the `ProcessingTask` table to
determine which parameter set should be used during processing. 

The `ProcessingTask` table is important for defining several important aspects of
downstream processing. Let's view the attributes to get a better understanding. 

In [ ]:
imaging.ProcessingTask.describe();

In [ ]:
imaging.ProcessingTask.heading

The `ProcessingParamSet` table adds two attributes that were not in the `Scan` table: 
+ `paramset_idx` 
+ `task_mode` 

The `paramset_idx` attribute is intended to help track
your image processing parameter sets. Suite2p, CaImAn, or EXTRACT
parameter sets can be inserted into this table. You can also choose the parameter sets on which you want
to run the image processing analysis. 

The `task_mode` attribute can be set to either
`load` or `trigger`. When set to `load`, running the processing step initiates a search
for exisiting output files of the image processing algorithm. When set to `trigger`, the
processing step will run a processing algorithm on the raw data. 

In [ ]:
imaging.ProcessingTask.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        processing_output_dir="subject1/session1/suite2p",
    )
)

Notice we are setting `task_mode` to `load`. Let's call populate on the `Processing`
table in the pipeline. 

In [ ]:
imaging.Processing.populate(**populate_settings)

While processing is complete in the step above, you can optionally curate the output of
image processing using the `Curation` table.

In [ ]:
imaging.Curation.describe();

In [ ]:
imaging.Curation.heading

In [ ]:
imaging.Curation.insert1(
    dict(
        **session_key,
        scan_id=0,
        paramset_idx=0,
        curation_id=0,
        curation_time="2021-04-30 12:22:15.032",
        curation_output_dir="subject1/session1/suite2p",
        manual_curation=False,
        curation_note="",
    )
)

Now, we'll populate several tables that store the output of image processing, including
`MotionCorrection`, `Segementation`, `Fluorescence`, and `Activity`.

Feel free to create new cells in this notebook to view each table's dependencies and
attributes. 

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

In [ ]:
imaging.Segmentation.populate(**populate_settings)

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

In [ ]:
imaging.Activity.populate(**populate_settings)

Now that data is in the DataJoint tables, the analysis workflows such as exploratory
analysis, alignment to behavior, statistical testing or modelling, and other downstream
analyses can be performed. 


### Query and fetch

The next section of this tutorial focuses on working with data that is already in the
database. 

DataJoint queries allow you to view and import data from the database into a python
variable using the `fetch()` method. 

There are several important features supported by `fetch()`:
+ By default, an empty `fetch()` imports a list of dictionaries containing all
  attributes of all entries in the table that is queried.
+ **`fetch1()`**, on the other hand, imports a dictionary containing all attributes of
  one of the entries in the table. By default, if a table has multiple entries,
  `fetch1()` imports the first entry in the table.
+ Both `fetch()` and `fetch1()` accept table attributes as an argument to query values
  of that particular attribute.
+ Recommended best practice is to **restrict** queries by primary key attributes of the
  table to ensure the accuracy of imported data. 
    + DataJoint offers a convenient way to fetch the primary key attributes of a table
      using `fetch("KEY")`.

Let's bring together these concepts of querying together by fetching a fluorescence
trace with `mask_id` of 10 into a variable `fluor_trace`.

In [ ]:
fluor_trace = (imaging.Fluorescence & "mask_id = '10'").fetch1("trace")

In [ ]:
average_image = (
    imaging.MotionCorrection.Summary & session_key & "field_idx=0"
).fetch1("average_image")

In [ ]:
mask_xpix, mask_ypix = (
    imaging.Segmentation.Mask * imaging.MaskClassification.MaskType
    & session_key
    & "mask_center_z=0"
    & "mask_npix > 130"
).fetch("mask_xpix", "mask_ypix")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mask_image = np.zeros(np.shape(average_image), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    mask_image[ypix, xpix] = True

In [ ]:
plt.imshow(average_image)
plt.contour(mask_image, colors="white", linewidths=0.5);

# Drop schemas

+ Schemas are not typically dropped in a production workflow with real data in it. 
+ At the developmental phase, it might be required for the table redesign.
+ When dropping all schemas is needed, the following is the dependency order.

In [ ]:
def drop_databases(databases):
    import pymysql.err
    conn = dj.conn()

    with dj.config(safemode=False):
        for database in databases:
            schema = dj.Schema(f'{dj.config["custom"]["database.prefix"]}{database}')
            while schema.list_tables():
                for table in schema.list_tables():
                    try:
                        conn.query(f"DROP TABLE `{schema.database}`.`{table}`")
                    except pymysql.err.OperationalError:
                        print(f"Can't drop `{schema.database}`.`{table}`. Retrying...")
            schema.drop()

# drop_databases(databases=['imaging_report', 'imaging', 'scan', 'session', 'subject', 'lab', 'reference'])